In [1]:
from glob import glob
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import plotly.graph_objs as go
from scipy.interpolate import splprep,splev
import math
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [51]:
def readExports(processParams):
    print('Importing mooring '+str(processParams.wbatParams.mooringNum))
    dfs = []
    for file in tqdm(glob(processParams.evParams.outputDir+'exports\\'+'*(regions).csv'),desc='Reading regions'):
        dfs.append(pd.read_csv(file))
    dfR = pd.concat(dfs)
    dfs = []
    for file in tqdm(glob(processParams.evParams.outputDir+'exports\\'+'*(targets).csv'),desc='Reading targets'):
        dfs.append(pd.read_csv(file))
    dfT = pd.concat(dfs)
    cols = np.append(dfT.columns[0:18],['Ping_number']).tolist()
    dfT = dfT[cols].drop(columns=['Region_name', 'Region_class'])
    dfR = dfR.drop(columns=['Region_name', 'Region_class','Target_length_mean'])
    dfR = dfR[dfR.TS_max < -30]
    dfT = dfT[dfT.Region_ID.isin(dfR.Region_ID)]
    return dfR, dfT

# Using Echoview terminology, so Major = X, Minor = Y
def targetSpline(df, processID, regionID):
    df = df[(df.Region_ID==regionID) & (df.Process_ID==processID)]
    tck, u  = splprep([df.Distance_major_axis, df.Distance_minor_axis,df.Target_true_depth],k=3)
    x_knots, y_knots, z_knots = splev(np.linspace(0,1,len(df.Distance_major_axis)-1), tck)
    return x_knots, y_knots, z_knots 
    
def trackPlot(df,processID, regionID, plotElements =['track_smoothed','track','track_direct','targets','track_linear']):
    # plotElements: list of elements to include:
    # targets: target locations
    # track: line between targets
    # track_smoothed: Cubic spline with equal resolution to the number of targets
    # track_direct: "flux" line from start to end target based on spline    
    x_knots, y_knots, z_knots = targetSpline(df, processID, regionID)
    track_smoothed = go.Scatter3d(x=x_knots, y=y_knots, z=z_knots,mode='lines',name='Smoothed Line')
    
    track_direct = go.Scatter3d(x=[x_knots[i] for i in (0, -1)], y=[y_knots[i] for i in (0, -1)], z=[z_knots[i] for i in (0, -1)],mode='lines',name='Direct Path')

    dfCur = df[(df.Region_ID==regionID) & (df.Process_ID==processID)]
    targets=go.Scatter3d(x=dfCur.Distance_major_axis, y=dfCur.Distance_minor_axis, z=dfCur.Target_true_depth,mode='markers',
        marker=dict(size=8,color='black', colorscale='plasma',opacity=0.8,colorbar=dict(title='TS',thickness=30),showscale=False),name='Targets')
    track = go.Scatter3d(x=dfCur.Distance_major_axis, y=dfCur.Distance_minor_axis, z=dfCur.Target_true_depth,mode='lines',name='Target Line')
        
     # This is the linear fit version of this, getting heading from the X/Y against ping number separately 
    x = dfCur.Ping_number
    X = sm.add_constant(x)
    y = dfCur.Distance_major_axis
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X) # make the predictions by the model
    dMaj = predictions.to_numpy()

    y = dfCur.Distance_minor_axis
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X) # make the predictions by the model
    dMin = predictions.to_numpy()

    y = dfCur.Target_true_depth
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X) # make the predictions by the model
    dZ = predictions.to_numpy()

    z_knots = [dZ[0],dZ[-1]]
    x_knots = [dMaj[0],dMaj[-1]]
    y_knots = [dMin[0],dMin[-1]]
    track_linear = go.Scatter3d(x=x_knots, y=y_knots, z=z_knots,mode='lines',name='Linear fit')
    
    allElements = {'track_smoothed':track_smoothed,'track':track,'track_direct':track_direct,'targets':targets,'track_linear':track_linear}
    fig = go.Figure(data=[allElements[i] for i in plotElements])
    split = (np.ceil(np.max([dfCur.Distance_major_axis.max() - dfCur.Distance_major_axis.min(),dfCur.Distance_minor_axis.max() - dfCur.Distance_minor_axis.min(),dfCur.Target_true_depth.max() - dfCur.Target_true_depth.min()]))/2)+.25
    fig.update_layout(showlegend=True,scene = dict(xaxis_title='Distance_major_axis',
                        yaxis_title='Distance_minor_axis',
                        zaxis_title='Depth'),
                        #xaxis = dict(nticks=4, range=[np.median(dfCur.Distance_major_axis)-split, np.median(dfCur.Distance_major_axis)+split]),
                        #yaxis = dict(nticks=4, range=[np.median(dfCur.Distance_minor_axis)-split, np.median(dfCur.Distance_minor_axis)+split]),
                        #zaxis = dict(nticks=4, range=[np.median(dfCur.Target_true_depth)-split, np.median(dfCur.Target_true_depth)+split])),
                        width=700,
                        margin=dict(r=20, b=10, l=10, t=10))

    fig.show()

    
def regionStats(dfR,dfT,processParams):
    x_knots, y_knots, z_knots = targetSpline(dfT, dfR.Process_ID, dfR.Region_ID)
    coords = [j for j in zip(x_knots,y_knots,z_knots)]
    d = np.diff(coords, axis=0)
    segdistsS = np.hypot(d[:,0], d[:,1],d[:,2])
    a = []
    for i in range(len(x_knots)-1):
        a.append(direction_lookup(processParams,x_knots[i+1],x_knots[i],y_knots[i+1],y_knots[i]))
    tDirS = np.mean(a)

    # This is the 'direct' spline version of this
    x_knots = [x_knots[i] for i in (0, -1)]
    y_knots = [y_knots[i] for i in (0, -1)]
    z_knots = [z_knots[i] for i in (0, -1)]
    coords = [j for j in zip(x_knots,y_knots,z_knots)]
    d = np.diff(coords, axis=0)
    segdistsD = np.hypot(d[:,0], d[:,1],d[:,2])
    tDirD = direction_lookup(processParams,x_knots[1],x_knots[0],y_knots[1],y_knots[0])
        
    # This is the linear fit version of this, getting heading from the X/Y plane
    dfCur = dfT[(dfT.Process_ID==dfR.Process_ID)&(dfT.Region_ID==dfR.Region_ID)]
    if dfCur.Distance_minor_axis.max()-dfCur.Distance_minor_axis.min() < dfCur.Distance_major_axis.max()-dfCur.Distance_major_axis.min():
        x = dfCur.Distance_major_axis
        X = sm.add_constant(x)
        y = dfCur.Distance_minor_axis
        model = sm.OLS(y, X).fit()
        predictions = model.predict(X) # make the predictions by the model

        x_knots = [X.Distance_major_axis.to_numpy()[0],X.Distance_major_axis.to_numpy()[-1]]
        y_knots = [predictions.to_numpy()[0],predictions.to_numpy()[-1]]
        z_knots = [dfCur.Target_true_depth.values[0],dfCur.Target_true_depth.values[-1]]
    
    
    elif dfCur.Distance_minor_axis.max()-dfCur.Distance_minor_axis.min() > dfCur.Distance_major_axis.max()-dfCur.Distance_major_axis.min():
        x = dfCur.Distance_minor_axis
        X = sm.add_constant(x)
        y = dfCur.Distance_major_axis
        model = sm.OLS(y, X).fit()
        predictions = model.predict(X) # make the predictions by the model

        y_knots = [X.Distance_minor_axis.to_numpy()[0],X.Distance_minor_axis.to_numpy()[-1]]
        x_knots = [predictions.to_numpy()[0],predictions.to_numpy()[-1]]
        z_knots = [dfCur.Target_true_depth.values[0],dfCur.Target_true_depth.values[-1]]
    
    tDirL = direction_lookup(processParams,x_knots[1],x_knots[0],y_knots[1],y_knots[0])
    
    coords = [j for j in zip(x_knots,y_knots,z_knots)]
    d = np.diff(coords, axis=0)
    segdistsL = np.hypot(d[:,0], d[:,1],d[:,2])
    
    tSpeedS=segdistsS.sum()/((dfT[(dfT.Process_ID == dfR.Process_ID)& (dfT.Region_ID == dfR.Region_ID)].Ping_number.max() - dfT[(dfT.Process_ID == dfR.Process_ID)& (dfT.Region_ID == dfR.Region_ID)].Ping_number.min())*.4)
    tSpeedD=segdistsD.sum()/((dfT[(dfT.Process_ID == dfR.Process_ID)& (dfT.Region_ID == dfR.Region_ID)].Ping_number.max() - dfT[(dfT.Process_ID == dfR.Process_ID)& (dfT.Region_ID == dfR.Region_ID)].Ping_number.min())*.4)
    tSpeedL=segdistsL.sum()/((dfT[(dfT.Process_ID == dfR.Process_ID)& (dfT.Region_ID == dfR.Region_ID)].Ping_number.max() - dfT[(dfT.Process_ID == dfR.Process_ID)& (dfT.Region_ID == dfR.Region_ID)].Ping_number.min())*.4)

    return tSpeedS, segdistsS.sum(), tDirS, tSpeedD, segdistsD.sum(), tDirD, tSpeedL, segdistsL.sum(), tDirL

def regionStats2(dfR,dfT,processParams):
    x_knots, y_knots, z_knots = targetSpline(dfT, dfR.Process_ID, dfR.Region_ID)
    coords = [j for j in zip(x_knots,y_knots,z_knots)]
    d = np.diff(coords, axis=0)
    segdistsS = np.hypot(d[:,0], d[:,1],d[:,2])
    a = []
    for i in range(len(x_knots)-1):
        a.append(direction_lookup(processParams,x_knots[i+1],x_knots[i],y_knots[i+1],y_knots[i]))
    tDirS = np.mean(a)

    # This is the 'direct' spline version of this
    x_knots = [x_knots[i] for i in (0, -1)]
    y_knots = [y_knots[i] for i in (0, -1)]
    z_knots = [z_knots[i] for i in (0, -1)]
    coords = [j for j in zip(x_knots,y_knots,z_knots)]
    d = np.diff(coords, axis=0)
    segdistsD = np.hypot(d[:,0], d[:,1],d[:,2])
    tDirD = direction_lookup(processParams,x_knots[1],x_knots[0],y_knots[1],y_knots[0])
        
    # This is the linear fit version of this, getting heading from the X/Y against ping number separately 
    dfCur = dfT[(dfT.Process_ID==dfR.Process_ID)&(dfT.Region_ID==dfR.Region_ID)]
    x = dfCur.Ping_number
    X = sm.add_constant(x)
    y = dfCur.Distance_major_axis
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X) # make the predictions by the model
    dMaj = predictions.to_numpy()

    y = dfCur.Distance_minor_axis
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X) # make the predictions by the model
    dMin = predictions.to_numpy()
    
    y = dfCur.Target_true_depth
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X) # make the predictions by the model
    dZ = predictions.to_numpy()

    z_knots = [dZ[0],dZ[-1]]
    x_knots = [dMaj[0],dMaj[-1]]
    y_knots = [dMin[0],dMin[-1]]
    
    tDirL = direction_lookup(processParams,x_knots[1],x_knots[0],y_knots[1],y_knots[0])
    
    coords = [j for j in zip(x_knots,y_knots,z_knots)]
    d = np.diff(coords, axis=0)
    segdistsL = np.hypot(d[:,0], d[:,1],d[:,2])
    
    tSpeedS=segdistsS.sum()/((dfT[(dfT.Process_ID == dfR.Process_ID)& (dfT.Region_ID == dfR.Region_ID)].Ping_number.max() - dfT[(dfT.Process_ID == dfR.Process_ID)& (dfT.Region_ID == dfR.Region_ID)].Ping_number.min())*.4)
    tSpeedD=segdistsD.sum()/((dfT[(dfT.Process_ID == dfR.Process_ID)& (dfT.Region_ID == dfR.Region_ID)].Ping_number.max() - dfT[(dfT.Process_ID == dfR.Process_ID)& (dfT.Region_ID == dfR.Region_ID)].Ping_number.min())*.4)
    tSpeedL=segdistsL.sum()/((dfT[(dfT.Process_ID == dfR.Process_ID)& (dfT.Region_ID == dfR.Region_ID)].Ping_number.max() - dfT[(dfT.Process_ID == dfR.Process_ID)& (dfT.Region_ID == dfR.Region_ID)].Ping_number.min())*.4)

    return tSpeedS, segdistsS.sum(), tDirS, tSpeedD, segdistsD.sum(), tDirD, tSpeedL, segdistsL.sum(), tDirL

def direction_lookup(processParams,destination_x, origin_x, destination_y, origin_y):
    deltaX = destination_x - origin_x
    deltaY = destination_y - origin_y
    degrees_temp = math.atan2(deltaX, deltaY)/math.pi*180
    # Flip for the inversion due to upward facing Xducer
    degrees_temp = 360-degrees_temp
    # Correct for the Xducer direction
    degrees_temp = (degrees_temp + processParams.wbatParams.mooringCompassOffset-360)
    # Whats the magnetic declination at the site?
    degrees_temp = degrees_temp + processParams.wbatParams.mooringMagDec
    # Convert check if its negative and correct
    if degrees_temp > 360:
        degrees_final = degrees_temp-360
    else:
        degrees_final = degrees_temp
    return degrees_final

def splineDF(dfR, dfT, processParams):
    #compassCor = {1:335,2:208,4:250,5:197,6:293}
    speedS,distS,headS = [],[],[]
    speedD,distD,headD = [],[],[]
    speedL,distL,headL = [],[],[]
    for index,row in tqdm(dfR.iterrows(),total=len(dfR),desc='Calculating splines'):
        sS,dS,hS,sD,dD,hD,sL,dL,hL = regionStats2(row,dfT,processParams)
        speedD.append(sD)
        distD.append(dD)
        headD.append(hD)
        speedS.append(sS)
        distS.append(dS)
        headS.append(hS)
        speedL.append(sL)
        distL.append(dL)
        headL.append(hL)
    dfR['directSpeed'] = np.around(np.array(speedD),3)
    dfR['directDist'] = np.around(np.array(distD),3)
    dfR['directHead'] = np.around(np.array(headD),3)
    dfR['smoothedSpeed'] = np.around(np.array(speedS),3)
    dfR['smoothedDist'] = np.around(np.array(distS),3)
    dfR['smoothedHead'] = np.around(np.array(headS),3)
    dfR['linearSpeed'] = np.around(np.array(speedL),3)
    dfR['linearDist'] = np.around(np.array(distL),3)
    dfR['linearHead'] = np.around(np.array(headL),3)
    return dfR

In [61]:
from mooringParams import paramsDAFT1 as processParams1 # Each mooring has a specific params file with 
dfR, dfT = readExports(processParams1)
dfRSpline = splineDF(dfR, dfT, processParams1)
dfRSpline.to_csv('../data/analysisFiles/TrackRegions'+str(processParams1.wbatParams.mooringNum)+'_linear2.csv')

from mooringParams import paramsDAFT2 as processParams2 # Each mooring has a specific params file with 
dfR, dfT = readExports(processParams2)
dfRSpline = splineDF(dfR, dfT, processParams2)
dfRSpline.to_csv('../data/analysisFiles/TrackRegions'+str(processParams2.wbatParams.mooringNum)+'_linear2.csv')

from mooringParams import paramsDAFT4 as processParams4 # Each mooring has a specific params file with 
dfR, dfT = readExports(processParams4)
dfRSpline = splineDF(dfR, dfT, processParams4)
dfRSpline.to_csv('../data/analysisFiles/TrackRegions'+str(processParams4.wbatParams.mooringNum)+'_linear2.csv')

from mooringParams import paramsDAFT5 as processParams5 # Each mooring has a specific params file with 
dfR, dfT = readExports(processParams5)
dfRSpline = splineDF(dfR, dfT, processParams5)
dfRSpline.to_csv('../data/analysisFiles/TrackRegions'+str(processParams5.wbatParams.mooringNum)+'_linear2.csv')

from mooringParams import paramsDAFT6 as processParams6 # Each mooring has a specific params file with 
dfR, dfT = readExports(processParams6)
dfRSpline = splineDF(dfR, dfT, processParams6)
dfRSpline.to_csv('../data/analysisFiles/TrackRegions'+str(processParams6.wbatParams.mooringNum)+'_linear2.csv')

Importing mooring 1



Importing mooring 2



Importing mooring 4



Importing mooring 5



Importing mooring 6


In [60]:
#i = 1232#14
trackPlot(dfT,1062,1235)